In [1]:
from pyspark import SparkContext
from pyspark import SQLContext
sc = SparkContext()
sqlContext = SQLContext(sc)
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.getOrCreate()

from pyspark.sql.functions import col, countDistinct
from pyspark.sql.functions import isnan, when, count, col

In [2]:
import pandas as pd

In [3]:
sdfTrain44 = sqlContext.read.csv("/gl-capstone-data/Team6-C-Sep/Data/train_store44.csv",header = True,inferSchema = True)

In [4]:
print(sdfTrain44.show(5))

+-----+-------------------+---------+--------+----------+-----------+
|   id|               date|store_nbr|item_nbr|unit_sales|onpromotion|
+-----+-------------------+---------+--------+----------+-----------+
|32292|2013-01-02 00:00:00|       44|  103520|      12.0|       null|
|32293|2013-01-02 00:00:00|       44|  103665|      20.0|       null|
|32294|2013-01-02 00:00:00|       44|  105574|      51.0|       null|
|32295|2013-01-02 00:00:00|       44|  105575|      74.0|       null|
|32296|2013-01-02 00:00:00|       44|  105577|      23.0|       null|
+-----+-------------------+---------+--------+----------+-----------+
only showing top 5 rows

None


In [5]:
sdfTrain44.count()

1913387

In [6]:
sdfTransactions44 = sqlContext.read.csv("/gl-capstone-data/Team6-C-Sep/Data/transactions_store44.csv",header = True,inferSchema = True)

In [7]:
print(sdfTransactions44.show(5))

+-------------------+---------+------------+
|               date|store_nbr|transactions|
+-------------------+---------+------------+
|2013-01-02 00:00:00|       44|        4821|
|2013-01-03 00:00:00|       44|        3618|
|2013-01-04 00:00:00|       44|        4169|
|2013-01-05 00:00:00|       44|        4921|
|2013-01-06 00:00:00|       44|        4925|
+-------------------+---------+------------+
only showing top 5 rows

None


In [8]:
sdfTransactions44.count()

1089

In [9]:
pdfTransactions44 = sdfTransactions44.toPandas()

In [10]:
pdfTransactions44.shape

(1089, 3)

In [11]:
pdfTransactions44.dtypes

date            datetime64[ns]
store_nbr                int64
transactions             int64
dtype: object

In [12]:
pdfTransactions44.head()

,date,store_nbr,transactions
0,2013-01-02,44,4821
1,2013-01-03,44,3618
2,2013-01-04,44,4169
3,2013-01-05,44,4921
4,2013-01-06,44,4925


In [13]:
lstAggregatedDailySales = sdfTrain44.groupby('date').sum("unit_sales").collect()

In [14]:
type(sdfTrain44)

pyspark.sql.dataframe.DataFrame

In [15]:
type(lstAggregatedDailySales)

list

In [16]:
lstAggregatedDailySales[0:10]

[Row(date=datetime.datetime(2015, 4, 26, 0, 0), sum(unit_sales)=37564.66099999999),
 Row(date=datetime.datetime(2015, 7, 14, 0, 0), sum(unit_sales)=31047.451000000005),
 Row(date=datetime.datetime(2015, 10, 10, 0, 0), sum(unit_sales)=50847.530000000006),
 Row(date=datetime.datetime(2013, 10, 31, 0, 0), sum(unit_sales)=13095.737000000005),
 Row(date=datetime.datetime(2014, 8, 4, 0, 0), sum(unit_sales)=25818.219000000016),
 Row(date=datetime.datetime(2013, 7, 8, 0, 0), sum(unit_sales)=23736.041999999998),
 Row(date=datetime.datetime(2013, 9, 22, 0, 0), sum(unit_sales)=29860.117000000006),
 Row(date=datetime.datetime(2015, 1, 8, 0, 0), sum(unit_sales)=22956.348000000024),
 Row(date=datetime.datetime(2014, 1, 20, 0, 0), sum(unit_sales)=47262.681000000026),
 Row(date=datetime.datetime(2014, 8, 26, 0, 0), sum(unit_sales)=22411.626)]

In [17]:
print lstAggregatedDailySales[0]["date"]

2015-04-26 00:00:00


In [18]:
print lstAggregatedDailySales[0]["sum(unit_sales)"]

37564.661


In [19]:
pdfAggregatedDailySales = pd.DataFrame([(lstAggregatedDailySales[0]["date"], lstAggregatedDailySales[0]["sum(unit_sales)"])], columns=("date", "unit_sales"))

In [20]:
pdfAggregatedDailySales

,date,unit_sales
0,2015-04-26,37564.661


In [21]:
for x in range(1,len(lstAggregatedDailySales)):
  pdfAggregatedDailySales.loc[x] =   [lstAggregatedDailySales[x]["date"], lstAggregatedDailySales[x]["sum(unit_sales)"]]

In [22]:
pdfAggregatedDailySales.head(10)

,date,unit_sales
0,2015-04-26,37564.661
1,2015-07-14,31047.451
2,2015-10-10,50847.530
3,2013-10-31,13095.737
4,2014-08-04,25818.219
5,2013-07-08,23736.042
6,2013-09-22,29860.117
7,2015-01-08,22956.348
8,2014-01-20,47262.681
9,2014-08-26,22411.626


In [23]:
len(lstAggregatedDailySales)

1089

In [24]:
len(pdfAggregatedDailySales)

1089

In [25]:
pdfAggregatedDailySales.dtypes

date          datetime64[ns]
unit_sales           float64
dtype: object

In [26]:
pdfResult = pd.merge(pdfTransactions44,
                 pdfAggregatedDailySales[['date', 'unit_sales']],
                 on='date', how = 'left')

In [27]:
pdfResult.head()

,date,store_nbr,transactions,unit_sales
0,2013-01-02,44,4821,30095.181
1,2013-01-03,44,3618,20447.057
2,2013-01-04,44,4169,22795.799
3,2013-01-05,44,4921,31382.508
4,2013-01-06,44,4925,36069.345


In [28]:
pdfResult.isnull().sum() 

date            0
store_nbr       0
transactions    0
unit_sales      0
dtype: int64

In [29]:
type(pdfResult)

pandas.core.frame.DataFrame

In [30]:
pdfResult.dtypes

date            datetime64[ns]
store_nbr                int64
transactions             int64
unit_sales             float64
dtype: object

In [31]:
pdfResult['date'] = pdfResult['date'].astype(str)

In [32]:
pdfResult.dtypes

date             object
store_nbr         int64
transactions      int64
unit_sales      float64
dtype: object

In [33]:
pdfResult.head()

,date,store_nbr,transactions,unit_sales
0,2013-01-02,44,4821,30095.181
1,2013-01-03,44,3618,20447.057
2,2013-01-04,44,4169,22795.799
3,2013-01-05,44,4921,31382.508
4,2013-01-06,44,4925,36069.345


In [35]:
sdfResult = sqlContext.createDataFrame(pdfResult)

In [36]:
type(sdfResult)

pyspark.sql.dataframe.DataFrame

In [37]:
sdfResult.show()

+----------+---------+------------+------------------+
|      date|store_nbr|transactions|        unit_sales|
+----------+---------+------------+------------------+
|2013-01-02|       44|        4821|         30095.181|
|2013-01-03|       44|        3618|20447.056999999997|
|2013-01-04|       44|        4169|22795.799000000003|
|2013-01-05|       44|        4921|31382.508000000005|
|2013-01-06|       44|        4925|         36069.345|
|2013-01-07|       44|        3871|         21025.329|
|2013-01-08|       44|        3593|         33993.257|
|2013-01-09|       44|        4006|         21296.042|
|2013-01-10|       44|        3323|15522.827999999998|
|2013-01-11|       44|        3873|19738.474999999995|
|2013-01-12|       44|        4423|         25418.678|
|2013-01-13|       44|        4676|31129.350000000002|
|2013-01-14|       44|        3450| 17518.41199999999|
|2013-01-15|       44|        3527|15761.720999999996|
|2013-01-16|       44|        4026|21994.872999999996|
|2013-01-1

In [38]:
sdfResult.count()

1089

In [39]:
sdfResult.printSchema()

root
 |-- date: string (nullable = true)
 |-- store_nbr: long (nullable = true)
 |-- transactions: long (nullable = true)
 |-- unit_sales: double (nullable = true)



In [40]:
sdfResult.write.csv("/gl-capstone-data/Team6-C-Sep/Data/train_transactions_daily_store44.csv", header=True)

In [41]:
sdfResultReload = sqlContext.read.csv("/gl-capstone-data/Team6-C-Sep/Data/train_transactions_daily_store44.csv",header = True,inferSchema = True)

In [42]:
sdfResultReload.count()

1089

In [43]:
sdfResultReload.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- store_nbr: integer (nullable = true)
 |-- transactions: integer (nullable = true)
 |-- unit_sales: double (nullable = true)



In [44]:
print(sdfResultReload.show(5))

+-------------------+---------+------------+-----------------+
|               date|store_nbr|transactions|       unit_sales|
+-------------------+---------+------------+-----------------+
|2014-07-02 00:00:00|       44|        4494|50616.44199999999|
|2014-07-03 00:00:00|       44|        3849|        34998.176|
|2014-07-04 00:00:00|       44|        4111|35662.77099999999|
|2014-07-05 00:00:00|       44|        4674|50965.70800000001|
|2014-07-06 00:00:00|       44|        5092|64146.52700000003|
+-------------------+---------+------------+-----------------+
only showing top 5 rows

None
